In [5]:
import os
os.environ['OPENAI_API_KEY'] = '' #secret openai api key to be added

In [2]:
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [3]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00


In [9]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)  # 0 = least creative/risky   1 = most creative/risky
name = llm("I want to open a restaurant for Pakistani food. Suggest a fancy name for this.")
print(name)



"Spice of Pakistan" or "Saffron Bites"


In [12]:
from langchain.prompts import PromptTemplate

#making template so every time we don't have to alter llm(string)
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

prompt_template_name.format(cuisine="Chinese")

'I want to open a restaurant for Chinese food. Suggest a fancy name for this.'

In [14]:
from langchain.chains import LLMChain  # For creating and managing chains of language model calls

chain = LLMChain( llm=llm, prompt=prompt_template_name )
chain.run("American")

'\n\n"Stateside Bistro"'

In [37]:
#Building simple sequentual chain, where input of second chain is output of first
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain = LLMChain( llm=llm, prompt=prompt_template_name )

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template= """Return some food menu item names for {restaurant_name} as a single list separated with commas only."""
)

food_items_chain = LLMChain( llm=llm, prompt=prompt_template_items )

In [39]:
from langchain.chains import SimpleSequentialChain

# first output(restaurant name) helps generate food items(second output)
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Italian")
print(response)



Margherita Pizza, Spaghetti Carbonara, Chicken Parmigiana, Fettuccine Alfredo, Tiramisu


In [43]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain = LLMChain( llm=llm, prompt=prompt_template_name, output_key="restaurant_name" )

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template= """Return some food menu item names for {restaurant_name} as a single list separated with commas only."""
)

food_items_chain = LLMChain( llm=llm, prompt=prompt_template_items, output_key="menu_items" )

In [46]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

chain({'cuisine':'Pakistani'})

{'cuisine': 'Pakistani',
 'restaurant_name': '\n"Spice Palace: Authentic Pakistani Cuisine"',
 'menu_items': '\n\nBiryani, Chicken Tikka Masala, Samosas, Haleem, Seekh Kabab, Karahi Gosht, Naan, Mango Lassi, Kheer, Chapli Kabab'}